<a href="https://colab.research.google.com/github/wallet-maker/spectra/blob/main/notebooks/Spectra_Colaboratory_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install packages

## install Spectra from pypi

In [ ]:
%pip install scSpectra

## or install Spectra from source

In [ ]:
!git clone https://github.com/dpeerlab/spectra

In [ ]:
!ls

In [ ]:
%cd spectra

In [ ]:
!pip install matplotlib==3.8.0

In [ ]:
%pip install -U matplotlib==3.8.0

# import relevant packages

In [ ]:
#import packages
import numpy as np
import json 
import scanpy as sc
from collections import OrderedDict
import scipy 
import pandas as pd
import matplotlib.pyplot as plt

#spectra imports 
import Spectra as spc
from Spectra import Spectra_util as spc_tl
from Spectra import K_est as kst
from Spectra import default_gene_sets


# Load gene_set_dictionary

**load a nested dictionary containing global and cell type specific gene sets in the following format:**

Gene set annotation dictionary with the keys being the celltypes (str) and values being dictionaries with gene set names as keys (str) and gene sets as values (lists of gene names/IDs which matches the gene names/IDs in adata.var_names).

For example:

```
gene_set_dictionary = {'celltype_1':{'gene_set_1':['gene_a', 'gene_b', 'gene_c'], 'gene_set_2':['gene_c','gene_a','gene_e','gene_f']},

'celltype_2':{'gene_set_1':['gene_a', 'gene_b', 'gene_c'], 'gene_set_3':['gene_a', 'gene_e','gene_f','gene_d']},

'celltype_3':{},

'global':"{'gene_set_4':['gene_m','gene_n']} #the global key must be supplied

```

**Note that one key in the dictionary must be 'global' with the corresponding value being a dictionary of gene sets which apply to all cells**

Spectra will use this dictionary to align factors to the input gene sets. Gene sets which apply to only one cell type in the data should be included in the dictionary of that cell type. If a gene sets applies to all cell types in the data, the gene set should be included in the dictionary for 'global'. If a gene set applies to more than one cell type but not all cell types in the data there are two options 1) Include this gene set in each cell type dictionary which will likely result in a separate factor for this gene set in each cell type. OR 2) include this gene set in the 'global' dictionary which will likely result in one factor for this gene set in all cell types. We give additional guidance on the advantages and disadvantages of either approach in the Supplementary Methods of the Spectra paper: https://doi.org/10.1101/2022.12.20.521311

**Load the default dictionary**
We used this dictionary to generate the results in the paper: https://doi.org/10.1101/2022.12.20.521311

In [ ]:
# load the default gene set dictionary from the Spectra paper:
annotations = spc.default_gene_sets.load()
annotations["global"]["all_wnt-beta-catenin-signaling"]

# Custom gene_set_dictionary

**Alternatively you can use Spectra with your custom gene set annotation dictionaries.**

Cytopus Knowledge Base

We supply the Cytopus knowledge base to construct custom input gene set dictionaries for Spectra. For a tutorial visit the github repository: https://github.com/wallet-maker/cytopus

**External Databases**

To obtain optimal results, we recommend using dedicated gene sets, like from the Cytopus database which are tailored to single cell RNA sequencing data (see Supplementary Methods for details https://doi.org/10.1101/2022.12.20.521311).

However, Spectra can also use gene set annotation dictionaries from external databases if they can be provided in the format described above.

# Load adata

In the Spectra paper we use a subsetted, anonymized, log1p-transformed (f(x) = ln(x+1), with x being the raw gene expression counts), and scran-normalized leukocyte single cell RNA sequencing data (https://doi.org/10.1101/2022.12.20.521311). While we recommend scran for leukocyte data, the method with log1p-transformed median library size normalized RNA sequencing data.

In [ ]:
# define data paths
obs_key = 'cell_type_annotations' #indicat the column name for the dataframe in adata.obs where to find the cell type lab

In [ ]:
# load adata
adata = spc.sample_data.load()
adata

In [ ]:
# alternatively indicate where to find the AnnData object (and uncomment below)
adata_path = '/content/spectra/data/sample_data.h5ad'
# adata = sc.read_h5ad(adata_path)

**Important: The cell type labels have to match with the cell type labels in the gene set dictionary**

In [ ]:
# cell type labels in adata
list(set(adata.obs[obs_key]))

In [ ]:
# cell type in gene set annotation dictionary
list(set(annotations.keys()))

**if labels do not match adjust the labels either in adata or in annotations**

For example define and map a dictionary mapping the annotations in the adata object to the keys in the gene set annotations dictionary or vice versa. **Note that if you have a cell type in your adata for which you do not have any gene sets in your gene set annotation dictionary you must include an empty dictionary under that cell type key.** Afterwards confirm that the matching has worked:

In [ ]:
#filter gene set annotation dict for genes contained in adata
annotations = spc_tl.check_gene_set_dictionary(
    adata,
    annotations,
    obs_key='cell_type_annotations',
    global_key='global')

# Fit Spectra model

**General comments**

We start by importing spectra. The easiest way to run spectra is to use the `est_spectra` function in the `spectra` module, as shown below. The default behavior is to set the number of factors equal to the number of gene sets plus one. However, this can be modified by passing an integer e.g. `L = 20` as an argument to the function or a dictionary that maps cell type to an integer per cell type. We provide a method for estimating the number of factors directly from the data by bulk eigenvalue matching analysis, which is detailed further below. 

**Computational comments**

Spectra is memory and cpu intensive.

The here-provided dataset can be run on most computers. For bigger datasets you want to run this in non-interactive mode, as runtime can be 1-2 days for a dataset of ca 200,000 cells with 250 factors and 10 cell types. Run time scales linearly with the number of cell types in your data.

An example hardware you could run a 200,000 cell, 10 cell type, 250 factor dataset on would be:

16 cpus Intel Xeon Gold 256 GB RAM

**Parameters**

`adata` : AnnData object containing cell_type_key with log count data stored in .X

`gene_set_dictionary`:dict or OrderedDict() maps cell types to gene set names to gene sets ; if use_cell_types == False then maps gene set names to gene sets ; must contain "global" key in addition to every unique cell type under .obs.<cell_type_key>

`L`: dict, OrderedDict(), int , NoneType number of factors per cell type ; if use_cell_types == False then int. Else dictionary. If None then match factors to number of gene sets (recommended)

`use_highly_variable` : bool if True, then uses highly_variable_genes

`cell_type_key`: str cell type key, must be under adata.obs.<cell_type_key> . If use_cell_types == False, this is ignored

`use_weights`: bool if True, edge weights are estimated based on graph structure and used throughout training

`lam`: float lambda parameter of the model. weighs relative contribution of graph and expression loss functions

`delta`: float delta parameter of the model. lower bounds possible gene scaling factors so that maximum ratio of gene scalings cannot be too large

`kappa`: float or None if None, estimate background rate of 1s in the graph from data

`rho`: float or None if None, estimate background rate of 0s in the graph from data

`use_cell_types` : bool if True then cell type label is used to fit cell type specific factors. If false then cell types are ignored

`n_top_vals` : int number of top markers to return in markers dataframe

`determinant_penalty` : float determinant penalty of the attention mechanism. If set higher than 0 then sparse solutions of the attention weights and diverse attention weights are encouraged. However, tuning is crucial as setting too high reduces the selection accuracy because convergence to a hard selection occurs early during training [todo: annealing strategy]

`filter_sets` : bool whether to filter the gene sets based on coherence

`label_factors` : bool whether to label the factors by their cell type specificity and their Szymkiewicz–Simpson overlap coefficient with the input marker genes

`overlap_threshold`: float minimum overlap coefficient to assign an input gene set label to a factor

``**kwargs`` : (num_epochs = 10000, lr_schedule = [...], verbose = False) arguments to .train(), maximum number of training epochs, learning rate schedule and whether to print changes in learning rate

**Returns**: SPECTRA_Model object [after training]

**In place**: adds 1. factors, 2. cell scores, 3. vocabulary, and 4. markers as attributes in .obsm, .var, .uns

**default parameters:** 

```
est_spectra(
    adata, 
    gene_set_dictionary, 
    L = None,
    use_highly_variable = True, 
    cell_type_key = None, 
    use_weights = True, 
    lam = 0.01, 
    delta=0.001,
    kappa = None, 
    rho = 0.001, 
    use_cell_types = True, 
    n_top_vals = 50, 
    filter_sets = True, 
    label_factors=True, 
    overlap_threshold= 0.2, 
    **kwargs)
```

**fit model with cell type annotations**

Running Spectra with the cell type annotations will give you both global and cell type specific factors.

In [ ]:
# fit the model (We will run this with only 2 epochs to decrease runtime in this tutorial)
model = spc.est_spectra(adata=adata, 
    gene_set_dictionary=annotations, 
    use_highly_variable=True,
    cell_type_key="cell_type_annotations", 
    use_weights=True,
    lam=0.1, # varies depending on data and gene sets, try between 0.5 and 0.001
    delta=0.001, 
    kappa=None,
    rho=0.001, 
    use_cell_types=True,
    n_top_vals=50,
    label_factors=True, 
    overlap_threshold=0.2,
    clean_gs = True, 
    min_gs_num = 3,
    num_epochs=2 #here running only 2 epochs for time reasons, we recommend 10,000 epochs for most datasets
)


In [ ]:
#explore eta parameter to detect new factors
model.return_eta_diag()

## Accessing quantities stored in AnnData

This function stores four important quantities in the AnnData, in addition to returning a fitted model object. *Factors* are the scores that tell you how much each gene contributes to each factor:

In [ ]:
#find the factors under adata.uns['SPECTRA_factors'] 
adata.uns['SPECTRA_factors']

In [ ]:
#so you can construct a dataframe for the factor gene weights

#include cell type specificity as a prefix into the index
index_labels = adata.uns['SPECTRA_overlap'].index
gene_weights = pd.DataFrame(adata.uns['SPECTRA_factors'], 
                            index= index_labels,
                            columns=adata.var[adata.var['spectra_vocab']].index)
gene_weights

*Markers* is an array of genes with top scores for every factor:

In [ ]:
#find an array of the marker genes per factor here:
adata.uns['SPECTRA_markers']

We also provide an approach to label the factors by their Szymkiewicz–Simpson overlap coefficient with the input gene sets. Each factors receives the label of the input gene set with the highest overlap coefficient, given that it the overlap coefficient is greater than the threshold defined in 'overlap_threshold'. Ties in the overlap coefficient by gene set size, selecting the label of the bigger gene set (because smaller gene sets might get bigger overlap coefficients by chance).

We provide a pandas.DataFrame indicating the overlap coefficients for each input gene set with each factor's marker genes. The index of this dataframe contains the *index* of each factor, *assigned label* as well as the *cell type specificity* for each factor in the format:

`['index' + '-X-' + 'cell type specificity' + '-X-' + 'assigned label', ...]`

We use `'-X-'` as a unique seperator to make string splitting and retrieval of the different components of the index easier.

In [ ]:
adata.uns['SPECTRA_overlap']

*Cell scores* are similarly the score of each factor for every cell:

In [ ]:
#find an array of cell scores per factor and cell here:
adata.obsm['SPECTRA_cell_scores']

Vocab is a boolean array that is `True` for genes that were used while fitting the model - note that this quantity is only added to the AnnData when `highly_variable` is set to `True`:

In [ ]:
#find the vocab here:
adata.var['spectra_vocab']

It also includes a dictionary of the factor numbers per cell type:

In [ ]:
#find the factor numbere dict here:
adata.uns['SPECTRA_L']

## Acessing model parameters

To access finer grained information about the model fit, we can look at the attributes of the model object directly. Model parameters can be accessed with functions associated with the model object

In [ ]:
#this is the model file
dir(model)

In [ ]:
#e.g. find the lambda parameter the model was trained with:
model.lam

Apart from cell scores and factors, we can also retrive a number of other parameters this way that are not by default added to the AnnData. Eta diag is the diagonal of the fitted factor-factor interaction matrix; however, its interpretation is that it measures the extent to which each factor is influenced by the prior information. In practice many of these values are zero, indicating that they are estimated without bias introduced by the annotation set. Eta is the full set of factor-factor interaction matrices, whose off diagonals measure the extent to which factors share the same genes. Rho and kappa are parameters that control the background rate of non-edges and edges respectively. These can be fixed throughout training (default) or estimated from the data by providing rho = None or kappa = None to the est_spectra() function or to model.train(). Finally gene scalings are correction factors that normalize each gene based on its mean expression value.

In [ ]:
model.return_eta_diag()

In [ ]:
adata.uns['SPECTRA_overlap']

Examine the parameters of the underlying internal model:

In [ ]:
model.internal_model

In [ ]:
#You can save the model like this (this way consumes a lot of storage but does not require the model parameters to load)
import pickle
with open('spectra_model.pickle', 'wb') as f:
    pickle.dump(model, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
#and load it like this:
with open('spectra_model.pickle', 'rb') as f:
    model = pickle.load(f)

In [ ]:
#this way needs less storage but requires the original adata, annotations and cell type annotations to load the model again
model.save('spectra_model_compact')

In [ ]:
#load the model (you will need all the parameters indicated here)
model = spc.load_from_pickle('spectra_model_compact',adata,gs_dict=annotations,cell_type_key='cell_type_annotations',
                     )

**fit model without cell type annotations**

You can also fit the model without using the cell type annotations. In this case, instead of a nested gene set annotation dictionary, supply a regular dictionary with gene set names as keys (str) and the gene sets as values (list of gene names/IDs which match the gene names/IDs in the adata.var_names)

In [ ]:
{'global':annotations['global']}

In [ ]:
#fit the model (We will run this with only 2 epochs to decrease runtime in this tutorial)
model_global = spc.est_spectra(adata = adata, gene_set_dictionary = annotations['global'], # because we do not use the cell types
                                                                                    # we will supply a regular dict
                                                                                    # instead of the nested dict above
                        use_highly_variable = True, cell_type_key =None, #"cell_type_annotations" ,
                        use_weights = True, lam = 0.1, 
                        delta=0.001,kappa = 0.00001, rho = 0.00001, 
                        use_cell_types = False, #set to False to not use the cell type annotations
                        n_top_vals = 25, 
                        clean_gs=True,
                        label_factors=True,
                        num_epochs=2 #for demonstration purposes we will only run 2 epochs, we recommend 10,000 epochs
                       )



In [ ]:
#show the labeled factors in the data:
adata.uns['SPECTRA_overlap'].index

In [ ]:
#visualize factor cell scores (this is poorly fitted bc we only ran 2 epochs)
factor_of_interest = adata.uns['SPECTRA_overlap'].index[80]
print('plotting factor:',adata.uns['SPECTRA_overlap'].index[80])

#add cell scores to obs
cell_scores = adata.obsm['SPECTRA_cell_scores'][:,0].astype(float)
adata.obs[factor_of_interest] = cell_scores
sc.pl.umap(adata,color=factor_of_interest,s=30,vmax=np.quantile(cell_scores,0.98))

In [ ]:
#save the adata
adata_save_path = 'adata_spectra.h5ad'#where to save the adata to
adata.write(adata_save_path)
print('Saved adata to:',adata_save_path)

# Accessing the fitted gene-gene graph

One outcome of fitting spectra is to fit a gene-gene graph where edges represent similarities between latent variables associated with each gene (a smoothed version of transcriptional similarity) To access this for say, TNK cells; use the following

In [ ]:
soft_graph = model_global.return_graph(ct = "global")
soft_graph

for large numbers of genes its clumsy to visualize the whole graph - to visualize a subgraph formed around a particular list of genes, use:

In [ ]:
gene_set = list(adata.uns['SPECTRA_markers'][0])
out = spc.graph_network(adata, soft_graph, gene_set)

#this will not show in Google Colaboratory but you can open the file manually outside Google Colaboratory
#out.show("test_graph.html")

In [ ]:
adata.uns['SPECTRA_markers'][0].shape

this will take N closest genes to your gene set and only visualize this subgraph. The interactive graph file gets saved as an html. To visualize multiple gene sets at the same time, we have a different version of the function that assigns a random color to each gene set:

In [ ]:
#gene sets is a list of lists
gene_set_2 = list(adata.uns['SPECTRA_markers'][1])
gene_sets = [gene_set,gene_set_2]
out = spc.graph_network_multiple(adata,soft_graph, gene_sets)
#out.show("test_graph.html")

# Fitting the model without AnnData

Instead of passing an AnnData object to est_spectra one can pass np.ndarray objects directly. The **kwargs contains arguments to the training function, lr_schedule = [1.0,.5,.1,.01,.001,.0001],num_epochs = 10000, verbose = False. To do this, initialize a model:

In [ ]:
adata

In [ ]:
#define number of factors (only est_spectra has the option to automatically set the factor numbers per celltype)

L = {}
for key in annotations.keys():
    length = len(list(annotations[key].values()))
    L[key] = length + 1
L

In [ ]:
#get the genes you want to use (for simplicity we will copy from the adata above)
vocab = list(adata.var_names)

In [ ]:
adata.X.todense().shape

In [ ]:
len(vocab)

In [ ]:
model = spc.SPECTRA_Model(X = adata.X.todense(), L=L, labels = np.array(adata.obs['cell_type_annotations']),  
                          gs_dict = annotations,lam = 0.01, vocab=vocab, use_weights=True,
                          delta=0.001,kappa = None, rho = 0.05,use_cell_types=True)
model.train(X = adata.X.todense(), labels = np.array(adata.obs['cell_type_annotations']),
            num_epochs=2) #we will only do 2 epochs here for demonstration purposes. We recommend 10,000 epochs for most datasets.

It is also required to run the model this way if you want to input arbitrary adjacency matrices instead of a dictionary of gene sets. The gene set dictionary is used to create an adjacency matrix when it is not None.

In [ ]:
#you will need the cell type containing model and vocab again --> for this tutorial we will retrain
model = spc.est_spectra(adata = adata, gene_set_dictionary = annotations, 
                        use_highly_variable = True, cell_type_key = "cell_type_annotations", 
                        use_weights = True, lam = 0.1, 
                        delta=0.001,kappa = 0.00001, rho = 0.00001, 
                        use_cell_types = True, n_top_vals = 25, 
                        label_factors = True, #whether to label the factors by their overlap coefficient with the input gene sets
                        overlap_threshold = 0.2, #minimum overlap coefficient that has to be surpassed to assign a label to a factor
                        num_epochs=2 #for demonstration purposes we will only run 2 epochs, we recommend 10,000 epochs
                       )

In [ ]:
#get an example adjacency matrix (the threshold should be properly determined based on the question)
threshold = 0.15
cell_types = list(set(adata.obs['cell_type_annotations'])) + ['global']
adj_matrix = {cell_type : (model.return_graph(ct = cell_type) > threshold).astype(float) for cell_type in cell_types}
adj_matrix

In [ ]:
dir(model)

In [ ]:
soft_graph

In [ ]:
adata.X.todense().shape

In [ ]:
adata.var['spectra_vocab'].value_counts()

In [ ]:
#train model with adjacency matrix
vocab = np.array(adata.var['spectra_vocab'][adata.var['spectra_vocab']].index)
X = adata.X.todense().T[adata.var['spectra_vocab']].T

model = spc.SPECTRA_Model(X = X, labels =  np.array(adata.obs['cell_type_annotations']),  
                          L = L, adj_matrix = adj_matrix, weights = None, lam = 0.01,  delta=0.001,kappa = None, rho = 0.05, 
                          use_cell_types = True)
model.train(X = X, labels =  np.array(adata.obs['cell_type_annotations']),
            num_epochs=2) #we will only do 2 epochs here for demonstration purposes. We recommend 10,000 epochs for most datasets.)

